## Experimenting with Fields

In [9]:

from sympy import Function, symbols, sqrt, init_printing
from sympy.vector import CoordSys3D 
import spb
import scipy.constants as const
import numpy as np
from random import seed, randint, randrange, choice, uniform
init_printing()


classes for points and systems (Not Recent)

In [10]:
class PointSystem:
    coordsys = CoordSys3D("c")
    magsyms = symbols("x y z")    
    unitsyms = (coordsys.i, coordsys.j, coordsys.k)
    r0 = 1
    def __init__(self, unitsyms=PointSystem.unitsyms, magsyms=PointSystem.magsyms):
        self._e = {0:unitsyms[0], 1:unitsyms[1], 2:unitsyms[2]}
        self._v = {0:magsyms[0], 1:magsyms[1], 2:magsyms[2]}
        self.dp = sum(self._v[x]*self.e[x] for x in range(2))
        self.objects = []
        self.has_charges = False
        self.has_masses = False
    def contain(self, *objs):
        if all(isinstance(obj, Point) for obj in objs):
            self.objects.append(*objs)
    





class Point:
    def __init__(self, system:PointSystem, position=None, **properties):
        self.r = position if position is not None else system.r0
        system.contain(self)
        if properties['q']:
            self.q = properties['q']
            system.has_charges = True
            self.E = electric_field(q=self.q, r=system.dp, r0=self.r)
        if properties['m']:
            self.m = properties['m']
            system.has_masses = True
        
        


NameError: name 'PointSystem' is not defined

In [1]:
R = CoordSys3D("c",)
E = Function('\vect{E}')
_pi, q, eps0 = symbols("pi q  epsilon_0")
x1,y1,z1 = symbols("x_1 y_1 z_1")

def randcuboidsurfpts(ranges=dict(x=[-2, 2], y=[-2, 2], z=[-2, 2]), num_points=500, seed_id=None, other=None):
    np.random.seed(seed_id)

    # Generate a random list of axes to decide which to keep constant
    axes = np.random.choice(['x', 'y', 'z'], num_points)
    axis_indices = {'x': axes == 'x', 'y': axes == 'y', 'z': axes == 'z'}

    # Initialize arrays to hold the point coordinates
    coords = {'x': np.zeros(num_points), 'y': np.zeros(num_points), 'z': np.zeros(num_points)}

    # Assign random values to the coordinates, with one axis being constant for each point
    for axis in axis_indices:
        # Select a random face for the constant axis
        coords[axis][axis_indices[axis]] = np.random.choice(ranges[axis], axis_indices[axis].sum())

        # For non-constant axes, assign random values within the range
        for other_axis in coords:
            if other_axis != axis:
                coords[other_axis][axis_indices[axis]] = np.random.uniform(*ranges[other_axis], axis_indices[axis].sum())
    if other is not None:
        for i in range(other):
            for k in ('x', 'y', 'z'):
                coords[k] = np.append(coords[k], np.random.uniform(*ranges[k]))
    # Convert arrays to tuples before returning
    return coords


r = x1*R.i + y1*R.j + z1*R.k
r0 = 0*R.i + 0*R.j + 0*R.k
eV = const.electron_volt

def electric_field(q,r,r0, const_symbols=True):
    ret = q/(4*_pi*eps0)*(r-r0)/(sqrt((r-r0).dot((r-r0))))**3
    if not const_symbols:
        ret = ret.subs({_pi:const.pi, eps0:const.epsilon_0})
    return ret

def vis_vector_field(field, s, ranges=dict(x=[-2, 2], y=[-2, 2], z=[-2, 2]), n=100):
    
    import k3d
    elef = field.subs(s).components
    elefs = [elef[R.i], elef[R.j], elef[R.k]]
    spb.graphics(
        spb.vector_field_3d(elefs[0], elefs[1], elefs[2], 
                            (x1, *ranges['x']),
                            (y1, *ranges['y']),
                            (z1, *ranges['z']),
                            use_cm=True,
                            streamlines=True,
                                    quiver_kw={"scale": 10000, "line_width": 0.05, "head_size": 0.5},
                            stream_kw=dict( starts=randcuboidsurfpts(ranges,
                                                                    num_points=n,
                                                                    other=500),
                                            width=0.001,
                                            color_map=k3d.colormaps.paraview_color_maps.Yellow___Gray___Blue
                                            )
                            ),
             backend=spb.KB,
             xlabel="x",
             ylabel="y",
             zlabel="z",
             legend=False,
             )

elef1 = electric_field(1000*eV, r, r0)
elef2 = electric_field(-1000*eV, r, r.subs({x1:1, y1:1, z1:1}))
elef3 = electric_field(-1000*eV, r, r.subs({x1:-0.1, y1:0.1, z1:0.1}))
elef4 = electric_field(10000*eV, r, r.subs({x1:0, y1:5, z1:0}))
relef = elef1+elef2+elef3+elef4

vis_vector_field(field=relef, s={_pi:const.pi, eps0:const.epsilon_0})

NameError: name 'CoordSys3D' is not defined